# Confidential ML Training Demo (Data Owner 2)

This notebook is the Data Owners part of the *Confidential ML Training Demo* showing how a simple logistic regression classifier can be trained while keeping the training data provably confidential. The demo requires the [Training Client API](https://github.com/decentriq/avato-python-client-training) and its dependencies to be installed.  

## 1 - Import dependencies and submission code

In [ ]:
import os
import pandas as pd
from avato import Client
from avato import Secret
from avato_training import Training_Instance
import example

dataowner2_username = os.getenv('DATAOWNER2_ID')
dataowner2_password = os.getenv('DATAOWNER2_PASSWORD')

# This is the hash of the code
expected_measurement = "8e6c77bd904826526918a7e8fd4ccdc35cb8f8b5e0241fc78749e80dedf00cdf"

# The datafile
dataowner2_file = "test-data/wine-dataowner2.csv"

backend_host = "avato.uksouth.cloudapp.azure.com" 
backend_port = 15005 

## 2 - Set instance id received from Analyst

In [ ]:
# TO MODIFY in accordance to Instance ID created and provided 
#   by the analyst (Analyst notebook, Section 2, Instance ID: [...])
instance_id_from_analyst = "9a1deb5e51b73ec56977230c19827b960280857caaf794ea1cda03ff91063658"

## 3 - Submit Data

#### Create client and connect to the instance created by the analyst

In [ ]:
# Create client
dataowner2_client = Client(
    username=dataowner2_username,
    password=dataowner2_password,
    instance_types=[Training_Instance],
    backend_host=backend_host,
    backend_port=backend_port
)

# Connect to instance (using ID from the analyst user)
dataowner2_instance = dataowner2_client.get_instance(instance_id_from_analyst)

#### Verify security and create own keypair

In [ ]:
# Verify security.
dataowner2_instance.validate_fatquote(
    expected_measurement=expected_measurement,
    accept_debug=True,
    accept_group_out_of_date=True
)

# Create and set public-private keypair for secure communication.
dataowner2_secret = Secret()
dataowner2_instance.set_secret(dataowner2_secret)

#### Get data format from the enclave

In [ ]:
# Get data format from the enclave
data_format = dataowner2_instance.get_data_format()
print("Data format:\n{}".format(data_format))

#### Load data

In [ ]:
# Load data
df = pd.read_csv(dataowner2_file)
df.head(2)

#### Submit data

In [ ]:
(ingested_rows, failed_rows) = dataowner2_instance.submit_data(df)
print("\nNumber of successfully ingested rows: {}, number of failed rows: {}".format(ingested_rows, failed_rows))